The main program that uses the database and defined python scripts to run the analysis

In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
scores = []
scores = list(map(runSimulation, range(40), repeat(seasonsData), repeat(qualiResultsData), repeat(driversData), 
                    repeat(constructorsData), repeat(enginesData)))
print(np.median(scores))

0.561282894736842


So, now when the data has been analysed, it's time to use it to predict the next qualification's results! In this simplified version, the grand prix is specified by the parameter circuit.

In [4]:
(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [5]:
newDrivers = json.load(open('newDrivers.json'))["drivers"]
#print(newDrivers)

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [6]:
circuit = 7

driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 5.0 %
	2: 15.7 %
	3: 23.0 %
	4: 21.3 %
	5: 18.1 %
	6: 14.3 %
	7: 0.6 %
	13: 0.1 %
	19: 0.5 %
	20: 1.4 %
Fernando Alonso (McLaren):
	7: 0.1 %
	8: 1.1 %
	9: 5.0 %
	10: 16.2 %
	11: 23.2 %
	12: 22.8 %
	13: 19.4 %
	14: 8.2 %
	15: 2.6 %
	16: 0.5 %
	17: 0.2 %
	19: 0.2 %
	20: 0.5 %
Kimi Räikkönen (Ferrari):
	1: 29.7 %
	2: 34.8 %
	3: 12.8 %
	4: 8.9 %
	5: 7.1 %
	6: 5.4 %
	7: 0.1 %
	19: 0.1 %
	20: 1.1 %
Sebastian Vettel (Ferrari):
	1: 59.4 %
	2: 20.0 %
	3: 8.1 %
	4: 4.7 %
	5: 4.1 %
	6: 2.8 %
	17: 0.1 %
	19: 0.2 %
	20: 0.6 %
Romain Grosjean (Haas F1 Team):
	8: 0.5 %
	9: 3.5 %
	10: 10.0 %
	11: 21.1 %
	12: 23.9 %
	13: 20.5 %
	14: 13.3 %
	15: 3.5 %
	16: 2.2 %
	17: 0.1 %
	18: 0.1 %
	19: 0.1 %
	20: 1.2 %
Nico Hülkenberg (Renault):
	5: 0.2 %
	6: 6.4 %
	7: 55.7 %
	8: 16.2 %
	9: 10.3 %
	10: 5.9 %
	11: 1.7 %
	12: 0.8 %
	13: 0.6 %
	14: 0.5 %
	16: 0.1 %
	18: 0.1 %
	19: 0.1 %
	20: 1.4 %
Sergio Pérez (Force India):
	5: 0.1 %
	6: 1.5 %
	7: 15.5 %
	8: 27.7 %
	9: 23.3 %
	10: 14.8 %
